In [4]:
import numpy as np
import scipy.linalg as slin
import networkx as nx
import timeit
import numpy as np
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.append(os.path.abspath('../benchmark_data/'))
sys.path.append(os.path.abspath('../'))
import tensorflow as tf
import benchmark_data_reader
from BNGPU import NOTEARS
from time import time
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd

In [5]:
benreader = benchmark_data_reader.BenchmarkReader()

In [ ]:
res = []

for i in benreader.get_dataset_name():
    data = benreader.read_data(i)
    W_true = data.W
    X = data.data.copy()
    #X = benchmark_data_reader.rank_transform(X)
    X = benchmark_data_reader.mean_var_normalize(X)
    d, n = data.num_gene, data.num_sample
    tic = time()
        
    with tf.device("/gpu:0"):
        tf.reset_default_graph()
        clf = NOTEARS.NoTearTF()
        clf.construct_graph(X)
    
    sess = tf.Session()
    sess.run(clf.graph_nodes['init_vars'])
    clf.model_train(sess)
        
    W_est = sess.run(clf.graph_nodes['weight'])
    time_span = time() - tic
    
    y_pred = np.abs(W_est.ravel())
    y_true = np.abs(W_true.ravel()) > 1e-5
    
    res.append((i, d, n, time() - tic, average_precision_score(y_true, y_pred), roc_auc_score(y_true, y_pred)))
    print(i, d, n, time() - tic, average_precision_score(y_true, y_pred), roc_auc_score(y_true, y_pred))
    
    #df_res = pd.DataFrame(res)
    #df_res.columns = ['experiment', 'num_gene', 'num_sample', 'runtime', 'aupr', 'roc']
    #df_res.to_csv('results/NOTEARS_benchmark_result.csv', index = False)

('syntern/DAG1_d100_n100', 100, 100, 26.555632829666138, 0.09465517954044071, 0.7872973725884861)
('syntern/DAG1_d100_n500', 100, 500, 25.552894115447998, 0.09004520694109334, 0.8178764705833144)
('syntern/DAG1_d100_n1000', 100, 1000, 25.73093819618225, 0.09968598386178396, 0.847401491936732)
